# Machine Learning Part
* goal of this notebook is to develop code for the machine learning portion
    * dataset loading
    * model
    * evaluation metrics
* code should be added into qdanalysis and refactored aferwards

In [1]:
import tensorflow as tf
from tensorflow import keras

from qdanalysis.models import adapt_resnet50

import pathlib

2024-02-14 15:13:26.808063: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-14 15:13:26.839627: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 15:13:26.839649: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 15:13:26.840442: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 15:13:26.845483: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
train_dir = './output/cerug_test_processing/CN'
test_dir = './output/cerug_test_processing/EN'

In [5]:
label_mode = 'categorical'
#shape image will be resized to (figure out ideal values later)
input_shape = (128, 128, 3)
seed = 42
validation_split = 0.2

#due to nature of data, the test and train splits are defined beforehand
train_data = keras.preprocessing.image_dataset_from_directory(train_dir, 
                                                              label_mode=label_mode, 
                                                              image_size=input_shape[:2], 
                                                              seed=seed, 
                                                              validation_split=validation_split, 
                                                              subset='training')
writer_classes = train_data.class_names

Found 18281 files belonging to 3 classes.
Using 14625 files for training.


In [4]:
writer_classes

['Writer0202', 'Writer0909']

In [5]:
validation_data = keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                   label_mode=label_mode,
                                                                   image_size=input_shape[:2],
                                                                   seed=seed,
                                                                   validation_split=validation_split,
                                                                   subset='validation')

Found 1930 files belonging to 2 classes.
Using 386 files for validation.


In [6]:
model = adapt_resnet50(input_shape, len(writer_classes))

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1000)              2049000   
                                                                 
 dense_1 (Dense)             (None, 2)                 2002      
                                                                 
Total params: 25,638,714
Trainable params: 2,051,002
Non-trainable params: 23,587,712
_________________________________________

In [7]:
# optimizer and loss taken from https://stackoverflow.com/questions/71704268/using-tf-keras-utils-image-dataset-from-directory-with-label-list
optimizer = tf.keras.optimizers.Nadam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07,
    name='Nadam'
)

lossfn = tf.keras.losses.CategoricalCrossentropy()

metrics = [
    keras.metrics.CategoricalAccuracy(),
    keras.metrics.Precision(),
    keras.metrics.Recall()
]

model.compile(optimizer=optimizer, loss=lossfn, metrics=metrics)

In [8]:
epochs = 10
history = model.fit(train_data, epochs=epochs, validation_data=validation_data)

Epoch 1/10


49/49 [==============================] - 29s 500ms/step - loss: 0.6940 - categorical_accuracy: 0.6159 - precision: 0.6159 - recall: 0.6159 - val_loss: 0.6522 - val_categorical_accuracy: 0.6580 - val_precision: 0.6580 - val_recall: 0.6580
Epoch 2/10
49/49 [==============================] - 23s 460ms/step - loss: 0.6491 - categorical_accuracy: 0.6418 - precision: 0.6418 - recall: 0.6418 - val_loss: 0.6423 - val_categorical_accuracy: 0.6788 - val_precision: 0.6788 - val_recall: 0.6788
Epoch 3/10
49/49 [==============================] - 22s 443ms/step - loss: 0.6258 - categorical_accuracy: 0.6535 - precision: 0.6535 - recall: 0.6535 - val_loss: 0.6392 - val_categorical_accuracy: 0.6684 - val_precision: 0.6684 - val_recall: 0.6684
Epoch 4/10
49/49 [==============================] - 22s 445ms/step - loss: 0.6090 - categorical_accuracy: 0.6716 - precision: 0.6716 - recall: 0.6716 - val_loss: 0.6338 - val_categorical_accuracy: 0.6762 - val_precision: 0.6762 - val_recall: 0.6762
Epoch 5/10
49/4